# Assignment 1.2: Word2vec preprocessing (20 points)

Preprocessing is not the most exciting part of NLP, but it is still one of the most important ones. Your task is to preprocess raw text (you can use your own, or [this one](http://mattmahoney.net/dc/text8.zip). For this task text preprocessing mostly consists of:

1. cleaning (mostly, if your dataset is from social media or parsed from the internet)
1. tokenization
1. building the vocabulary and choosing its size. Use only high-frequency words, change all other words to UNK or handle it in your own manner. You can use `collections.Counter` for that.
1. assigning each token a number (numericalization). In other words, make word2index и index2word objects.
1. data structuring and batching - make X and y matrices generator for word2vec (explained in more details below)

**ATTN!:** If you use your own data, please, attach a download link. 

Your goal is to make **Batcher** class which returns two numpy tensors with word indices. It should be possible to use one for word2vec training. You can implement batcher for Skip-Gram or CBOW architecture, the picture below can be helpful to remember the difference.

![text](https://raw.githubusercontent.com/deepmipt/deep-nlp-seminars/651804899d05b96fc72b9474404fab330365ca09/seminar_02/pics/architecture.png)

There are several ways to do it right. Shapes could be `x_batch.shape = (batch_size, 2*window_size)`, `y_batch.shape = (batch_size,)` for CBOW or `(batch_size,)`, `(batch_size, 2*window_size)` for Skip-Gram. You should **not** do negative sampling here.

They should be adequately parametrized: CBOW(window_size, ...), SkipGram(window_size, ...). You should implement only one batcher in this task; and it's up to you which one to chose.

Useful links:
1. [Word2Vec Tutorial - The Skip-Gram Model](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/)
1. [Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/pdf/1301.3781.pdf)
1. [Distributed Representations of Words and Phrases and their Compositionality](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf)

You can write the code in this notebook, or in a separate file. It can be reused for the next task. The result of your work should represent that your batch has a proper structure (right shapes) and content (words should be from one context, not some random indices). To show that, translate indices back to words and print them to show something like this:

```
text = ['first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including']

window_size = 2

# CBOW:
indices_to_words(x_batch) = \
        [['first', 'used', 'early', 'working'],
        ['used', 'against', 'working', 'class'],
        ['against', 'early', 'class', 'radicals'],
        ['early', 'working', 'radicals', 'including']]

indices_to_words(labels_batch) = ['against', 'early', 'working', 'class']
```

In [39]:
from collections import Counter
from nltk.corpus import stopwords

import numpy as np
import nltk

nltk.download("stopwords")

class Batcher():
    def __init__(self, data, window_size, batch_size = 2):
        NUM_WORDS = 15000
        STOP_WORDS = set(nltk.corpus.stopwords.words("english"))
        UNIQ_TOKEN = "<UNQ>"
        
        data = [word for word in data if word not in STOP_WORDS]
        most_common = Counter(data)
        counts = Counter(data)

        self.w2i = {}
        self.i2w = {}
        for index, word in enumerate(counts.most_common(NUM_WORDS)):
            self.w2i.update({word[0] : index})
            self.i2w.update({index : word[0]})
        self.i2w.update({-1 : UNIQ_TOKEN})
        self.w2i.update({UNIQ_TOKEN : -1})

        self.window_size = window_size
        self.batch_size = batch_size
        self.data = [self.w2i[word] if word in self.w2i else -1 for word in data]
        
    def __iter__(self):
        return self
    
    def __next__(self):
        label_ind = []
        label = []
        batch = []
        for _ in range(self.batch_size):
            index = int(np.random.uniform(0,len(self.data)))
            if index == 0 or (len(data) - index - 1 < self.window_size):
                index = self.window_size
            label_ind.append(index)
        for ind in label_ind:
            label.append(self.data[ind])
            list_ = [i for i in range(-self.window_size, self.window_size+1) if i!=0]
            b = [self.data[ind + i] for i in list_]
            batch.append(b)
        return batch, label

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/nmakarov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [40]:
with open('./text8') as file:
        data = file.read().split()

In [44]:
%%time
a = Batcher(data, 3)

CPU times: user 16.5 s, sys: 151 ms, total: 16.6 s
Wall time: 16.6 s
